In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import os
import pytz

import MetaTrader5 as mt5

from datetime import datetime
from pathlib import Path
from scipy import stats

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [2]:
# mt5 directory
mt5dir = r"C:\Program Files\MetaTrader 5 IC Markets (SC) - 1"
# mt5dir = r"C:\Program Files\Pepperstone MetaTrader 5"
mt5loc = Path(mt5dir) / 'terminal64.exe'

# connect to MetaTrader 5
if not mt5.initialize(str(mt5loc)):
    print("initialize() failed")
    mt5.shutdown()
    
# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2982, maxbars=100000000, codepage=0, ping_last=254849, community_balance=0.0, retransmission=0.0, company='Raw Trading Ltd', name='MetaTrader 5 IC Markets (SC)', language='English', path='C:\\Program Files\\MetaTrader 5 IC Markets (SC) - 1', ...)
(500, 2982, '23 Jun 2021')


In [3]:
# Get the symbols from IC Markets MT5, excluding equity stocks
symbols = mt5.symbols_get("*, !*.*")

product_columns = [
    'name',
    'description',
    'path',
    'currency_base',
    'currency_profit',
    'currency_margin',
    'bid',
    'ask',
    'spread',
#     'last',
    'trade_contract_size',
    'volume_min',
    'volume_max',
    'volume_step',
    'trade_tick_value',
    'trade_tick_size',
    'margin_initial',
#     'margin_maintenance',
    'swap_long',
    'swap_short'
]


product_df = pd.DataFrame(columns=product_columns)

for symbol in symbols:
    product_df = product_df.append(
        pd.Series(
            [
                symbol.name,
                symbol.description,
                symbol.path,
                symbol.currency_base,
                symbol.currency_profit,
                symbol.currency_margin,
                symbol.bid,
                symbol.ask,
                symbol.ask - symbol.bid,
#                 symbol.last,
                symbol.trade_contract_size,
                symbol.volume_min,
                symbol.volume_max,
                symbol.volume_step,
                symbol.trade_tick_value,
                symbol.trade_tick_size,
                symbol.margin_initial,
#                 symbol.margin_maintenance,
                symbol.swap_long,
                symbol.swap_short
            ],
            index=product_columns
        ),
        ignore_index=True
    )

product_df

,name,description,path,currency_base,currency_profit,currency_margin,bid,ask,spread,trade_contract_size,volume_min,volume_max,volume_step,trade_tick_value,trade_tick_size,margin_initial,swap_long,swap_short
0,EURUSD,Euro vs US Dollar,Forex\Majors\EURUSD,EUR,USD,EUR,1.18260,1.18260,0.00000,100000.0,0.01,200.0,0.01,1.000000,0.00001,100000.0,-5.23,-0.47
1,GBPUSD,Great Britain Pound vs US Dollar,Forex\Majors\GBPUSD,GBP,USD,GBP,1.38262,1.38267,0.00005,100000.0,0.01,200.0,0.01,1.000000,0.00001,100000.0,-3.44,-2.73
2,USDCHF,US Dollar vs Swiss Franc,Forex\Majors\USDCHF,USD,CHF,USD,0.91514,0.91517,0.00003,100000.0,0.01,200.0,0.01,1.092693,0.00001,100000.0,0.11,-4.87
3,USDJPY,US Dollar vs Japanese Yen,Forex\Majors\USDJPY,USD,JPY,USD,109.90700,109.90900,0.00200,100000.0,0.01,200.0,0.01,0.909844,0.00100,100000.0,-1.72,-3.94
4,USDCAD,US Dollar vs Canadian Dollar,Forex\Majors\USDCAD,USD,CAD,USD,1.25381,1.25385,0.00004,100000.0,0.01,200.0,0.01,0.797544,0.00001,100000.0,-2.95,-2.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,EURBBL_U1,Euro Bobl - September 21 CFD,Bonds CFDs\EURBBL_U1,USD,EUR,EUR,134.64000,134.65000,0.01000,100.0,1.00,100.0,1.00,1.182600,0.01000,0.0,0.00,0.00
133,EURBND_U1,Euro Bund - September 21 CFD,Bonds CFDs\EURBND_U1,USD,EUR,EUR,174.56000,174.57000,0.01000,100.0,1.00,100.0,1.00,1.182600,0.01000,0.0,0.00,0.00
134,EURSCA_U1,Euro Schatz - September 21 CFD,Bonds CFDs\EURSCA_U1,USD,EUR,EUR,112.18000,112.19000,0.01000,100.0,1.00,100.0,1.00,1.182600,0.01000,0.0,0.00,0.00
135,ITB10Y_U1,Euro BTP Italian 10 YR - September 21 CFD,Bonds CFDs\ITB10Y_U1,USD,EUR,EUR,152.95000,152.97000,0.02000,100.0,1.00,100.0,1.00,1.182600,0.01000,0.0,0.00,0.00


In [6]:
product_df.to_clipboard()

In [4]:
product_df[product_df['volume_min'] != product_df['volume_step']]

,name,description,path,currency_base,currency_profit,currency_margin,bid,ask,spread,trade_contract_size,volume_min,volume_max,volume_step,trade_tick_value,trade_tick_size,margin_initial,swap_long,swap_short
89,XRPUSD,Ripple US Dollar,Crypto\XRPUSD,USD,USD,USD,0.6084,0.6255,0.0171,1.0,1.0,100.0,0.01,0.00010,0.00010,0.0,-20.0,-20.0
90,EOSUSD,EOS US Dollar,Crypto\EOSUSD,USD,USD,USD,3.8070,3.8460,0.0390,1.0,1.0,100.0,0.01,0.00010,0.00010,0.0,-20.0,-20.0
91,EMCUSD,Emercoin US Dollar,Crypto\EMCUSD,USD,USD,USD,0.0001,0.1778,0.1777,1.0,1.0,100.0,0.01,0.00010,0.00010,0.0,-20.0,-20.0
92,NMCUSD,NameCoin US Dollar,Crypto\NMCUSD,USD,USD,USD,0.0050,7.9000,7.8950,1.0,1.0,100.0,0.01,0.00100,0.00100,0.0,-20.0,-20.0
93,PPCUSD,PeerCoin US Dollar,Crypto\PPCUSD,USD,USD,USD,0.8450,7.1880,6.3430,1.0,1.0,100.0,0.01,0.00100,0.00100,0.0,-20.0,-20.0
129,ADAUSD,Cardano (USD),Crypto\ADAUSD,USD,USD,USD,0.0000,0.0000,0.0000,1.0,1.0,100.0,0.01,0.00001,0.00001,0.0,-22.5,-15.0


In [5]:
product_df['name']

0         EURUSD
1         GBPUSD
2         USDCHF
3         USDJPY
4         USDCAD
         ...    
132    EURBBL_U1
133    EURBND_U1
134    EURSCA_U1
135    ITB10Y_U1
136     Sbean_Q1
Name: name, Length: 137, dtype: object

In [6]:
product_df['currency_profit'].unique()

array(['USD', 'CHF', 'JPY', 'CAD', 'NZD', 'GBP', 'AUD', 'SGD', 'DKK',
       'HKD', 'NOK', 'PLN', 'SEK', 'TRY', 'ZAR', 'CNH', 'CZK', 'HUF',
       'MXN', 'RUB', 'THB', 'EUR'], dtype=object)

In [35]:
product_df.to_clipboard()

# Download OHLC Data

In [8]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
timeframe = mt5.TIMEFRAME_D1
lookback = 10000000

product_ohlc = {}

for ticker in product_df['name']:
    product_ohlc[ticker] = pd.DataFrame(mt5.copy_rates_from_pos(ticker, timeframe, 0, lookback))
    try:
        product_ohlc[ticker]['time'] = pd.to_datetime(product_ohlc[ticker]['time'], unit='s')
        product_ohlc[ticker].set_index('time', inplace=True)
    except KeyError:
        print(f'KeyError on {ticker}')

product_ohlc['EURUSD']

KeyError on EURUSD
KeyError on GBPUSD
KeyError on USDCHF
KeyError on USDJPY
KeyError on USDCAD
KeyError on AUDUSD
KeyError on AUDNZD
KeyError on AUDCAD
KeyError on AUDCHF
KeyError on AUDJPY
KeyError on CHFJPY
KeyError on EURGBP
KeyError on EURAUD
KeyError on EURCHF
KeyError on EURJPY
KeyError on EURNZD
KeyError on EURCAD
KeyError on GBPCHF
KeyError on GBPJPY
KeyError on CADCHF
KeyError on CADJPY
KeyError on GBPAUD
KeyError on GBPCAD
KeyError on GBPNZD
KeyError on NZDCAD
KeyError on NZDCHF
KeyError on NZDJPY
KeyError on NZDUSD
KeyError on USDSGD
KeyError on AUDSGD
KeyError on CHFSGD
KeyError on EURDKK
KeyError on EURHKD
KeyError on EURNOK
KeyError on EURPLN
KeyError on EURSEK
KeyError on EURSGD
KeyError on EURTRY
KeyError on EURZAR
KeyError on GBPDKK
KeyError on GBPNOK
KeyError on GBPSEK
KeyError on GBPSGD
KeyError on GBPTRY
KeyError on NOKJPY
KeyError on NOKSEK
KeyError on SEKJPY
KeyError on SGDJPY
KeyError on USDCNH
KeyError on USDCZK
KeyError on USDDKK
KeyError on USDHKD
KeyError on 

""


In [259]:
for ticker, ohlc in product_ohlc.items():
    ohlc.to_csv(f'data/{ticker}.csv')

# Calculate Volatility

In [18]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

    # alternative caltulation using ewma
#     return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [19]:
# Create an empty DataFrame to store score

ins_risk_columns = [
    'ticker',
    'last_date',
    'ins_risk'
]

ins_risk_table = pd.DataFrame(columns=ins_risk_columns)

# How many (series) candles back for std dev calculation?
vola_window = 24

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, ohlc in product_ohlc.items():
    try:
        ins_risk = volatility(ohlc['close'], vola_window) * 16
        last_date = ohlc.index[-1]
        ins_risk_table = ins_risk_table.append(
            pd.Series(
                [
                    ticker,
                    last_date,
                    ins_risk
                ],
                index=ins_risk_columns
            ),
            ignore_index=True
        )
    except KeyError:
        print(f'KeyError on {ticker}')

ins_risk_table

KeyError on ADAUSD
KeyError on TRXUSD
KeyError on XMRUSD


,ticker,last_date,ins_risk
0,EURUSD,2021-07-02,0.056902
1,GBPUSD,2021-07-02,0.069191
2,USDCHF,2021-07-02,0.069109
3,USDJPY,2021-07-02,0.054362
4,USDCAD,2021-07-02,0.075771
...,...,...,...
130,EURBBL_U1,2021-07-02,NaN
131,EURBND_U1,2021-07-02,NaN
132,EURSCA_U1,2021-07-02,NaN
133,ITB10Y_U1,2021-07-02,NaN


In [269]:
ins_risk_table.to_clipboard()

In [20]:
product_df['ins_risk'] = ins_risk_table['ins_risk']

In [5]:
product_df['tick_value_min'] = product_df['trade_contract_size'] * product_df['volume_min'] * product_df['trade_tick_value'] * product_df['trade_tick_size']
product_df['ins_risk_curr'] = product_df['bid'] * product_df['tick_value_min'] * product_df['ins_risk'] * product_df['trade_contract_size']
product_df

KeyError: 'ins_risk'